In [ ]:
import nltk
import random
import re
import string

# Laboratório de Modelos de Linguagem

## Prof. André Carvalho

## Implementando Modelos de Linguagem Probabilisticos

Hoje vamos ver o passo a passo da implementação de um modelo de linguagem baseado em trigramas, com smoothing de Laplace.

Como sempre em NLP, vamos começar pegando um corpus e pre-processar.

Usaremos o corpus do machado, mas não apenas Dom Casmurro

In [ ]:
def limpa(raw):
    return [re.sub('\W+\s*', ' ',i).lstrip() for i in raw.lower().split('.') if i != '']

In [ ]:
#limpa("Testando, a limpeza. O que será que ela faz? 1.2 vai separar ou não?")

print(nltk.corpus.machado.readme())

In [ ]:
s = nltk.corpus.machado.readme()
livros = re.findall(r"(\w+\/\w+\.txt):",s)
corpus = []
for t in livros:
    raw = nltk.corpus.machado.raw(t)
    corpus+= limpa(raw)
    
for i in range(len(corpus)):
    corpus[i] = '<s> '+corpus[i]+' </s>'
print(corpus[200],len(corpus))

Vamos agora fazer uma função que recebe um corpus como uma lista  e retorna um vocabulário com todos os tokens com frequência maior que `freq`.

Ele vai ser um dicionário tendo como chave o termo e valor a frequência (vai facilitar).

Pode usar Counter no lugar do dicionário também.

In [ ]:

def vocabularioFixo(corpus,freq):
    tk = nltk.tokenize.WhitespaceTokenizer()
    tokens = tk.tokenize(' '.join(corpus))
    freqs = nltk.FreqDist(tokens)
    voc = {i[0]:i[1] for i in freqs.items() if i[1]>=freq}
    unk = sum([i[1] for i in freqs.items() if i[1]<freq])
    voc['<UNK>'] = unk
    return voc

voc = vocabularioFixo(corpus,10)

#print(voc['<UNK>'])
print(corpus[0:10],vocabularioFixo(corpus[0:10],2))


De posse do vocabulário, agora é hora de gerar os unigramas e suas probabilidades.

Lembrando, a probabilidade de um unigrama é:

$$ P(w_i) = \frac{freq(w_i)}{tottokens} $$

Palavras fora do vocabulário __devem__ ser contadas, mas devem ficar na probabilidade de `UNK`. Vamos modificar o vocabulário anterior para contar os `UNK`:

In [ ]:
def unigrama(corpus,vocabulario):
    # O vocabulário já tem as frequencias
    freqtot = sum(vocabulario.values())
    return {x:vocabulario[x]/freqtot for x in vocabulario.keys()}

modeluni = unigrama(corpus[0:10],voc)
modeluni


Vamos agora fazer um modelo de bigramas baseado neste mesmo vocabulário.

Lembrando:

$$P(w_i|w_{i-1}) = \frac{f(w_{i-1},w_{i})}{f(w_{i-1})} $$



In [ ]:
from collections import Counter

def bigrama(corpus,vocabulario):
    bigramas = Counter()
    for s in corpus:
        tk = nltk.tokenize.WhitespaceTokenizer()
        tokens = tk.tokenize(s)
        big = list(nltk.bigrams(tokens))
        for i in range(len(big)):
            if big[i][0] not in vocabulario:
                big[i] = ('<UNK>',big[i][1])
            elif big[i][1] not in vocabulario:
                big[i] = (big[i][0],'<UNK>')
        bigramas.update(big)
    return {x:bigramas[x]/vocabulario[x[0]] for x in bigramas}
            
        
bigrama(corpus,voc)[('dom','casmurro')]


Esse modelo está sem smoothing.

Vamos modificar ele para receber um $\lambda$ que vai representar o quanto de frequencia vai ser adicionado a todos os bigramas.

Se este valor for 1, vai ser o que vimos na aula passada. Mas podemos colocar mais baixo. Então fica:

$$ P_l(w_i|w_{i-1}) = \frac{f(w_i|w_{i-1})+\lambda}{f(w_{i-1})+|V|\lambda} $$

Vamos fazer uma função que retorna a probabilidade de um bigrama nunca antes visto:

In [ ]:
from collections import defaultdict

def bigrama(corpus,vocabulario,lamb=0.01):
    bigramas = Counter()
    for s in corpus:
        tk = nltk.tokenize.WhitespaceTokenizer()
        tokens = tk.tokenize(s)
        big = list(nltk.bigrams(tokens))
        for i in range(len(big)):
            lbig = list(big[i])
            if lbig[0] not in vocabulario:
                lbig[0] = '<UNK>'
            if lbig[1] not in vocabulario:
                lbig[1] = '<UNK>'
            big[i] = tuple(lbig)
        bigramas.update(big)
    probs = {}
    bigramas = dict(bigramas)
    for x in bigramas:
        if x[0] not in probs:
            probs[x[0]]= {}
        probs[x[0]][x[1]] = (bigramas[x]+lamb)/(vocabulario[x[0]]+len(vocabulario)*lamb)
                
    return probs,bigramas

modelbig, freqbig = bigrama(corpus,voc,0.01)
print(modelbig['dom'])

Também precisamos de uma função para calcular a probabilidade residual dos bigramas que nunca foram vistos:

In [ ]:
 
def bigramadesc(big,vocabulario,lamb):
    return lamb/(vocabulario[big[0]]+len(vocabulario)*lamb)



bigramadesc('dom',voc,0.1)

Vamos agora implementar o modelo de trigramas.

Para trigramas podemos usar uma função built-in da NLTK, `trigrams`, ou uma função genérica `ngrams`.

In [ ]:
def trigrama(corpus,vocabulario,freqbig,lamb=0.01):
    trigramas = Counter()
    for s in corpus:
        tk = nltk.tokenize.WhitespaceTokenizer()
        tokens = tk.tokenize(s)
        tri = list(nltk.trigrams(tokens))
        for i in range(len(tri)):
            ltri = list(tri[i])
            for j in range(len(ltri)):
                if ltri[j] not in vocabulario:
                    ltri[j] = '<UNK>'
            tri[i] = tuple(ltri)
        trigramas.update(tri)
    probs = {}
    tregramas = dict(trigramas)
    for x in trigramas:
        if (x[0],x[1]) not in probs:
            probs[(x[0],x[1])]= {}
        probs[(x[0],x[1])][x[2]] = (trigramas[x]+lamb)/(freqbig[(x[0],x[1])]+len(vocabulario)*lamb)
                
    return probs,trigramas
                
    return probs
modeltri,freqtri = trigrama(corpus,voc,freqbig,0.001)#[('rio','de','janeiro')]
modeltri[('rio','de')]

In [ ]:
freqbig[('rio','de')]
#freqtri[('rio','de','janeiro')]

De posse do modelo, vamos fazer um gerador agora?

Um gerador sequencial:
 - Gera um \<s\>
 - Procura por uma palavra inicial, sendo um bigrama começando por \<s\> + probabilidade unigrama da palavra.
 - A partir da segunda palavra, já usa unigrama+bigrama+trigrama
    
Vamos fazer um gerador de modelo de bigramas:

In [ ]:

#funcao q recebe uma distribuicao e escolhe um valor aleatoriamente seguindo a distribuicao
def escolhe(dist):
    prob = random.random()
    v = 0
    for i in dist:
        v+=dist[i]
        if(v>prob):
            return dist[i],i
    return dist[i],i

def escolhaGulosa(dist):
    r = max(dist,key=dist.get)
    return dist[r],r

def escolheTopk(dist,k):
    prob = random.random()
    v = 0
    l = {x:dist[x] for x in sorted(dist, key=dist.get, reverse=True)[:k]}
    topk = {x:l[x]/sum(l.values()) for x in l}
    #gerar topk
    for i in topk:
        v+=topk[i]
        if(v>prob):
            return topk[i],i
    return topk[i],i


anterior = '<s>'
_,atual = escolheTopk(modelbig[anterior],20)
while(atual!='</s>'):
    print(atual,end=' ')
    _,nova = escolheTopk(modeltri[(anterior,atual)],20)
    anterior = atual
    atual = nova

    


Vamos fazer agora um interpolando uni/bi com uma interpolação simples (0.9 pra bigrama, 0.1 pra unigrama)

In [ ]:
def escolhe(dist):
    prob = random.random()
    v = 0

    for i in dist:
        if(v>prob):
            return dist[i],i
        v+=dist[i]
    return dist[i],i

def gerador(modeluni,modelbi,modeltri,voc,lamb,lista = True):
    saida = ['<s>']
    probab = random.random()
    dist = {x:modeluni[x]*0.1 + 0.9* bigramadesc(('<s>',x),voc,lamb) for x in modeluni}
    dist.update({x:0.1*modeluni[x] + 0.9*modelbi['<s>'][x] for x in modelbi['<s>']})
    print(sum(dist.values()))    
    palavra = escolhe(dist)[1]
    while(palavra!='</s>'):
        print(palavra, end=' ')
        dist = {x:modeluni[x]*0.1 + 0.9* bigramadesc((palavra,x),voc,lamb) for x in modeluni}
        dist.update({x:0.1*modeluni[x] + 0.9*modelbi[palavra][x] for x in modelbi[palavra]})
        palavra = escolhe(dist)[1]
gerador(modeluni,modelbig,modeltri,voc,0.01)

# Computando a qualidade do modelo

Vamos computar a perplexidade para ver a qualidade dos nossos modelos.

Relembrando, a perplexidade é:

$$ \sqrt[N]{\prod_{i=1}^{N}\frac{1}{P(w_i|w_{i-k}...w_{i-1})}}$$

In [ ]:
def perplexidade(probsent, N):
    return 1/probsent**(1/N) # onde N eh o tamanho das distribuições do modelo/numero de tokens
# Se for em espaco log, eh e^((-1/N)*soma(log P(w_i)))

def probsent(modelo,corpus):
    return 

def perplexidadeModelo(modelo,corpus):
    return